In [2]:
#image net classifiation is a world competition categorizwe 1000 images that held where new cnn network win
#and every year we get new cnn

#in transfer learrning we use state of art algorithum that win and have best accuracy

# so we will use that networks or model

#vg16 is the best

from keras.layers import Input, Lambda, Dense, Flatten#flatten used at last layer number of outputs i have so here i have 4 and that created by dense layerfrom keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16 # here i will use transfer learnning so i will cut last layer bcz it have thousad of ouput so i will use 4 oupts 
from keras.applications.vgg16 import preprocess_input 
from keras.preprocessing import image#bcz our input is image
from keras.preprocessing.image import ImageDataGenerator# help in image augmentatin help in generating new image like zooming in out
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]



In [5]:
train_path = 'train'
test_path = 'test'

In [4]:

#remove vgg16 last layer  
#image size 3 for number of channel becauae currently i have rgb 
#black and white 1 chanel
# add preprocessing layer to the front of VGG

#winer of 2012 and 2016 use vgg16
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)#include_top=False delete furst and last last layer getting to remove now i can use this project

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [10]:
#don't have to train states because already are train 
# don't train existing weights


for layer in vgg.layers:
    layer.trainable = False
  

In [11]:
  # useful for getting number of classes
folders = glob('train/*')

In [12]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

#for appending number of category at last layer

In [13]:
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)#simply telling number of outputers at last layer

In [14]:

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)



In [15]:
# view the structure of the model
model.summary() 

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [16]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [18]:
# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator
#just generating more dataset
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [20]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5216 images belonging to 2 classes.


In [22]:
test_set = test_datagen.flow_from_directory('test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)


Epoch 1/5
162/163 [============================>.] - ETA: 17s - loss: 0.2065 - accuracy: 0.9236

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# save it as a h5 file

import tensorflow as tf

from keras.models import load_model

model.save('model_vgg16.h5')

In [ ]:
# Similary you can use the same template for Vgg 19, Resnet50, Mobilenet. All you have to import the library. Below are the examples

from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3





# validating model

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
from keras.application.vgg16 import preprocess_input
import numpy as np

model = load_model('model_vgg16.h5')
img = image.load_img('val/pnenumunia', rtarget_size = (224, 224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
img_data = preprocess_input(x)
classes = model.predict(img,data)